# Sentiment Analysis

## 1. data load

In [1]:
SOURCE_FILEPATH = "C:\\Users\\hello\\Desktop\\interview\\"
OUTPUT_PATH = "C:\\Users\\hello\\Desktop\\result\\"

In [2]:
import pandas as pd
import os
from afinn import Afinn
from konlpy.tag import Okt
from textblob import TextBlob
from googletrans import Translator
import re
import time

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
afinn = Afinn(emoticons=True)

In [4]:
def file_list_in_directory(path, extension='.csv'):
    file_list = os.listdir(path)
    csv_file_list = [file for file in file_list if file.endswith(extension)]
    return csv_file_list

In [5]:
def save_file(filepath: str, df):
    df.to_csv(filepath, sep=',', encoding='utf-8-sig')

In [6]:
def load_file(source_path):
    print(source_path)
    df = pd.read_csv(source_path)
    return df

In [7]:
def get_eng_score_afinn(s):
    s = str(s)
    try:
        return afinn.score(s)
    except:
        print("Score Error ", s)
        return 0

In [8]:
LETTERS = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
def text_cleaning(s):
    return LETTERS.sub('', str(s))    

In [9]:
file_list = file_list_in_directory(SOURCE_FILEPATH)

## google trans api

In [10]:
def translator_api(text):
    try:
        translator = Translator()
        comment = translator.translate(text=text, dest='en').text
        return comment
    except Exception as e:
        return ""

## google trans page using selenium

In [10]:
def ready_to_translate_driver():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("https://translate.google.com/#view=home&op=translate&tl=en")
    driver.find_element_by_xpath('//*[@id="sugg-item-ko"]').click()
    driver.implicitly_wait(3)
    
    driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]').click()
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[4]/div[1]/div[2]/div[2]').click()
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    driver.implicitly_wait(3)
    return driver

def translate(driver, source_text):
    try:
        elem = driver.find_element_by_xpath("//*[@id=\"source\"]")
        elem.clear()
        elem.send_keys(source_text)
        time.sleep(1)
        driver.implicitly_wait(3)
        result = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/div[3]/div[1]/div[2]/div/span[1]')
        return result.text
    except:
        print(source_text)
        return source_text

In [11]:
driver = ready_to_translate_driver()

def sentimental_using_afinn(filename):
    print(filename)
    df = load_file(SOURCE_FILEPATH + filename)
    score_list = list()
    translated_list = list()
    for idx, row in df.iterrows():
        comment = text_cleaning(row['contents'])
        try:
            if row['hangul'] == True:
                time.sleep(1)
                comment = translate(driver, comment)
                time.sleep(1)
        finally:
            translated_list.append(comment)
            score_list.append(get_eng_score_afinn(comment))
        
    df.insert(4, 'translate', translated_list)
    df.insert(5, 'score(afinn)', score_list)
    save_file(OUTPUT_PATH + filename, df)

[WDM] - Current google-chrome version 83.0.4103
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/83.0.4103.39/chromedriver_win32.zip


[WDM] - Unpack archive C:\Users\hello\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.zip


In [12]:
for filename in file_list:
    sentimental_using_afinn(filename)

zico-2.csv
C:\Users\hello\Desktop\interview\zico-2.csv
zico-3.csv
C:\Users\hello\Desktop\interview\zico-3.csv


In [1]:
import pandas as pd

In [40]:
name = "zico"
path1 = "C://Users/hello/Desktop/interview/" + name + "-1.csv"
path2 = "C://Users/hello/Desktop/interview/" + name + "-2.csv"
path3 = "C://Users/hello/Desktop/interview/" + name + "-3.csv"

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)

df = pd.concat([df1, df2, df3])
df = df.drop(df.columns[[0]], axis=1)

output_path = "C://Users/hello/Desktop/result/"
df.to_csv(output_path + name +".csv", mode='w')

,author,contents,likeCount,date,translate,score(afinn),hangul
0,ace spades,She's so pretty,0,2020-05-06T06:25:42.000Z,She's so pretty,1.0,False
1,jeikey,Like si vienes de maristas,0,2020-05-05T08:49:28.000Z,Like si vienes de maristas,2.0,False
2,max flynn,She is soooo ICONIC ❤️❤️❤️❤️❤️,0,2020-05-05T00:37:37.000Z,She is soooo ICONIC ❤️❤️❤️❤️❤️,0.0,False
3,Jayy T,Am I the only one who's still shocked that tha...,0,2020-05-03T23:39:54.000Z,Am I the only one who's still shocked that tha...,-2.0,False
4,Kosar Beiki,How about BTS?? Weren't they the first after b...,0,2020-05-03T23:02:54.000Z,How about BTS?? Weren't they the first after b...,0.0,False
...,...,...,...,...,...,...,...
599,Secret K,Love her she so humble love you Zach,0,2015-10-31T05:52:41.000Z,Love her she so humble love you Zach,7.0,False
600,Hannah Naumann,Love Zach and Ariana interviews,29,2015-10-31T04:03:24Z,Love Zach and Ariana interviews,3.0,False
601,abigail kidimbu,mom💗,6,2015-10-31T03:51:37Z,mom,0.0,False
602,Ashley Stephens,yay you mentioned aaron and liz c':,1,2015-10-31T03:51:02Z,yay you mentioned aaron and liz c':,0.0,False
